In [ ]:
import re
import spacy
from textblob import TextBlob

# Charger le modèle linguistique en anglais
nlp = spacy.load("en_core_web_sm")

def extract_feature_clauses(text, features):
    # Utiliser une expression régulière pour diviser le texte à n'importe quel signe de ponctuation, "but", ou "however" (insensible à la casse)
    sentences = re.split(r'[,.!?]|(?:\s(?:but|however)\s)', text, flags=re.IGNORECASE)

    # Analyser chaque phrase avec spaCy
    extracts_per_feature = {feature: set() for feature in features}
    for sentence in sentences:
        doc = nlp(sentence)
        for feature in features:
            # Vérifier si la caractéristique est mentionnée dans la phrase
            if any(token.text.lower() == feature.lower() for token in doc):
                extracts_per_feature[feature].add(sentence.strip())

    return extracts_per_feature

def analyze_sentiment(text):
    # Utiliser TextBlob pour l'analyse de sentiment
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity

    # Classer le sentiment en positif, négatif ou neutre
    if sentiment > 0:
        return "positif"
    elif sentiment < 0:
        return "négatif"
    else:
        return "neutre"

def process_reviews(reviews, features, feature_name):
    all_feature_extracts = set()

    for review in reviews:
        feature_extracts = extract_feature_clauses(review, features)
        for feature in features:
            all_feature_extracts.update(feature_extracts[feature])

    print(f"Sentiment des extraits pour {feature_name} :")
    for extract in all_feature_extracts:
        sentiment = analyze_sentiment(extract)
        print(f"  - {extract} : {sentiment}")

    positive_counts = 0
    negative_counts = 0

    for extract in all_feature_extracts:
        sentiment = analyze_sentiment(extract)
        if sentiment == "positif":
            positive_counts += 1
        elif sentiment == "négatif":
            negative_counts += 1

    print(f"\nPourcentage d'avis positifs/négatifs pour {feature_name} :")
    total_reviews = positive_counts + negative_counts
    if total_reviews > 0:
        percentage_positive = (positive_counts / total_reviews) * 100
        percentage_negative = (negative_counts / total_reviews) * 100
        print(f"  - Positif {percentage_positive:.2f}%, Négatif {percentage_negative:.2f}%")
    else:
        print("  - Aucun avis")

# Exemple d'utilisation avec une liste de mots-clés pour la batterie, le prix, et la connectivité
reviews = [
    "The battery life is ok; it lasts the whole day. However, the charge doesn't last long enough.",
    "I hate the battery longevity; it's much worse than my old device. The product is fine blablalalalallalalal blalallalalla.",
    "It is expensive compared to other options on the market. However, the connectivity features are impressive.",
    "The device has excellent connectivity, allowing seamless integration with other devices. But the connectivity issues need improvement.",
    "I have a problem with the battery",
]

battery_features = ["battery", "charge", "longevity", "battery life"]
price_features = ["price", "cost", "affordable","expensive"]
connectivity_features = ["connectivity", "connection", "network"]

# Appeler la fonction pour le traitement de la batterie
process_reviews(reviews, battery_features, "la batterie")

# Appeler la fonction pour le traitement du prix
process_reviews(reviews, price_features, "le prix")

# Appeler la fonction pour le traitement de la connectivité
process_reviews(reviews, connectivity_features, "la connectivité")


Sentiment des extraits pour la batterie :
  - I have a problem with the battery : neutre
  - The battery life is ok; it lasts the whole day : positif
  - the charge doesn't last long enough : négatif
  - I hate the battery longevity; it's much worse than my old device : négatif

Pourcentage d'avis positifs/négatifs pour la batterie :
  - Positif 33.33%, Négatif 66.67%
Sentiment des extraits pour le prix :
  - It is expensive compared to other options on the market : négatif

Pourcentage d'avis positifs/négatifs pour le prix :
  - Positif 0.00%, Négatif 100.00%
Sentiment des extraits pour la connectivité :
  - The device has excellent connectivity : positif
  - the connectivity issues need improvement : neutre
  - the connectivity features are impressive : positif

Pourcentage d'avis positifs/négatifs pour la connectivité :
  - Positif 100.00%, Négatif 0.00%


In [ ]:
import re
import spacy
from transformers import BertTokenizer, BertForSequenceClassification

# Charger le modèle linguistique en anglais
nlp = spacy.load("en_core_web_sm")

def extract_feature_clauses(text, features):
    # Utiliser une expression régulière pour diviser le texte à n'importe quel signe de ponctuation, "but", ou "however" (insensible à la casse)
    sentences = re.split(r'[,.!?]|(?:\s(?:but|however)\s)', text, flags=re.IGNORECASE)

    # Analyser chaque phrase avec spaCy
    extracts_per_feature = {feature: set() for feature in features}
    for sentence in sentences:
        doc = nlp(sentence)
        for feature in features:
            # Vérifier si la caractéristique est mentionnée dans la phrase
            if any(token.text.lower() == feature.lower() for token in doc):
                extracts_per_feature[feature].add(sentence.strip())

    return extracts_per_feature

def analyze_sentiment_with_bert(text):
    # Tokeniser le texte
    inputs = tokenizer(text, return_tensors="pt")

    # Faire une prédiction avec le modèle BERT
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = logits.argmax().item()

    # Remapper les classes selon vos besoins
    sentiment_mapping = {0: "Négatif", 1: "Négatif", 2: "Neutre", 3: "Positif", 4: "Positif"}
    predicted_sentiment = sentiment_mapping[predicted_class]

    # Vérifier si des mots négatifs spécifiques sont présents dans le texte
    for word in negative_words:
        if word.lower() in text.lower():
            predicted_sentiment = "Négatif"
            break

    return predicted_sentiment

# Charger le modèle BERT pré-entraîné pour l'analyse de sentiment
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

def process_reviews_with_bert(reviews, features, feature_name):
    all_feature_extracts = set()

    for review in reviews:
        feature_extracts = extract_feature_clauses(review, features)
        for feature in features:
            all_feature_extracts.update(feature_extracts[feature])

    print(f"Sentiment des extraits pour {feature_name} :")
    for extract in all_feature_extracts:
        sentiment = analyze_sentiment_with_bert(extract)
        print(f"  - {extract} : {sentiment}")

    positive_counts = 0
    negative_counts = 0

    for extract in all_feature_extracts:
        sentiment = analyze_sentiment_with_bert(extract)
        if sentiment == "Positif":
            positive_counts += 1
        elif sentiment == "Négatif":
            negative_counts += 1

    print(f"\nPourcentage d'avis positifs/négatifs pour {feature_name} :")
    total_reviews = positive_counts + negative_counts
    if total_reviews > 0:
        percentage_positive = (positive_counts / total_reviews) * 100
        percentage_negative = (negative_counts / total_reviews) * 100
        print(f"  - Positif {percentage_positive:.2f}%, Négatif {percentage_negative:.2f}%")
    else:
        print("  - Aucun avis")

# Exemple d'utilisation avec une liste de mots-clés pour la batterie, le prix, et la connectivité
reviews = [
    "The battery life is ok; it lasts the whole day. However, the charge doesn't last long enough.",
    "I hate the battery longevity; it's much worse than my old device. The product is fine.",
    "The cost is expensive compared to other options on the market. However, the connectivity features are impressive.",
    "The device has excellent connectivity, allowing seamless integration with other devices. But the connectivity issues need improvement.",
    "the battery is problematic",
    "there is a damage in the charge"
]

battery_features = ["battery", "charge", "longevity", "battery life"]
price_features = ["price", "cost", "affordable","cheap","expensive"]
connectivity_features = ["connectivity", "connection", "network"]

# Liste de mots négatifs spécifiques
negative_words = ["issue", "problem", "damage"]

# Appeler la fonction pour le traitement de la batterie avec BERT
process_reviews_with_bert(reviews, battery_features, "la batterie")

# Appeler la fonction pour le traitement du prix avec BERT
process_reviews_with_bert(reviews, price_features, "le prix")

# Appeler la fonction pour le traitement de la connectivité avec BERT
process_reviews_with_bert(reviews, connectivity_features, "la connectivité")


Sentiment des extraits pour la batterie :
  - the battery is problematic : Négatif
  - the charge doesn't last long enough : Négatif
  - there is a damage in the charge : Négatif
  - The battery life is ok; it lasts the whole day : Neutre
  - I hate the battery longevity; it's much worse than my old device : Négatif

Pourcentage d'avis positifs/négatifs pour la batterie :
  - Positif 0.00%, Négatif 100.00%
Sentiment des extraits pour le prix :
  - The cost is expensive compared to other options on the market : Neutre

Pourcentage d'avis positifs/négatifs pour le prix :
  - Aucun avis
Sentiment des extraits pour la connectivité :
  - the connectivity issues need improvement : Négatif
  - The device has excellent connectivity : Positif
  - the connectivity features are impressive : Positif

Pourcentage d'avis positifs/négatifs pour la connectivité :
  - Positif 66.67%, Négatif 33.33%
